# 세팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 경로이동
import os

os.chdir("/content/drive/MyDrive/컴퓨터비전/프로젝트/parking-space-counter")

In [ ]:
!dir

parking-space-counter.ipynb


# util.py


In [ ]:
import pickle

from skimage.transform import resize
import numpy as np
import cv2

EMPTY = True
NOT_EMPTY = False

MODEL = pickle.load(open("model.p", "rb"))

def empty_or_not(spot_bgr):
  flat_data = []

  img_resized = resize(spot_bgr, (15, 15, 3))
  flat_data.append(img_resized.flatten())
  flat_data = np.array(flat_data)

  y_output = MODEL.predict(flat_data)

  if y_output == 0:
    return EMPTY
  else:
    return NOT_EMPTY


def get_parking_spots_bboxes(connected_components):
  (totalLabels, label_ids, values, centroid) = connected_components # 튜플 언패킹: 네 개의 값을 별도의 변수에 할당

  slots = []
  coef = 1

  # 0번째는 배경이므로 제외
  for i in range(1, totalLabels):
    # cv2.CC_STAT_LEFT: 바운딩 박스의 왼쪽 상단 x좌표
    # cv2.CC_STAT_TOP: 바운딩 박스의 왼쪽 상단 y좌표
    # cv2.CC_STAT_WIDTH: 바운딩 박스의 너비
    # cv2.CC_STAT_HEIGHT: 바운딩 박스의 높이
    x1 = int(values[i, cv2.CC_STAT_LEFT] * coef)
    y1 = int(values[i, cv2.CC_STAT_TOP] * coef)
    w = int(values[i, cv2.CC_STAT_WIDTH] * coef)
    h = int(values[i, cv2.CC_STAT_HEIGHT] * coef)

    slots.append([x1, y1, w, h])

  return slots

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.1.3 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# main.py

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# from util import get_parking_spots_bboxes, empty_or_not

def calc_diff(im1, im2):
  # np.mean: 평균값 계산
  return np.abs(np.mean(im1) - np.mean(im2))

mask = 'mask_1920_1080.png'
video_path = 'parking_1920_1080_loop.mp4'
output_video = 'output_video.mp4'

mask = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

# 연결된 구성 요소를 분석하고 각 구성 요소에 대한 통계 정보 제공
# 분석할 이미지, 연결성(4 or 8), 레이블 이미지의 데이터 타입(32비트 정수형)
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)
# 총 구성 요소의 수, 레이블, (x, y, width, height, area) 정보, 중심좌표 (x, y)

spots = get_parking_spots_bboxes(connected_components)

spots_status = [None for j in spots]
diffs = [None for j in spots]

previous_frame = None

frame_nmr = 0
ret = True
step = 30
while ret:
  ret, frame = cap.read()

  if frame_nmr % step == 0 and previous_frame is not None:
    for spot_index, spot in enumerate(spots):
      x1, y1, w, h = spot

      spot_crop = frame[y1:y1+h, x1:x1+w, :]

      diffs[spot_index] = calc_diff(spot_crop, previous_frame[y1:y1+h, x1:x1+w, :])
    print([diffs[j] for j in np.argsort(diffs)][::-1])

  if frame_nmr % step == 0:
    if previous_frame is None:
      arr_ = range(len(spots))
    else:
      arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]

    for spot_index in arr_:
      spot = spots[spot_index]
      x1, y1, w, h = spot

      spot_crop = frame[y1:y1+h, x1:x1+w, :]

      spot_status = empty_or_not(spot_crop)

      spots_status[spot_index] = spot_status

  if frame_nmr % step == 0:
    previous_frame = frame.copy()

  for spot_index, spot in enumerate(spots):
    spot_status = spots_status[spot_index]
    x1, y1, w, h = spots[spot_index]

    if spot_status:
      processed_frame = cv2.rectangle(frame, (x1, y1), (x1+w, y1+h), (0, 255, 0), 2)
    else:
      processed_frame = cv2.rectangle(frame, (x1, y1), (x1+w, y1+h), (0, 0, 255), 2)

  cv2.rectangle(processed_frame, (80, 20), (550, 80), (0, 0, 0,), -1)
  cv2.putText(processed_frame, 'Available spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

  frame_nmr += 1

  out.write(processed_frame)

cap.release()
out.release()

Output hidden; open in https://colab.research.google.com to view.